In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow.keras import layers
import time
from IPython import display
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
from pathlib import Path
import numpy as np
from PIL import Image

ce code prépare les données NotMNIST pour l'entraînement d'un modèle de réseau de neurones en les chargeant, les divisant en ensembles d'entraînement et de test, et les prétraitant pour les rendre appropriées pour un modèle CNN.


In [ ]:


# Spécifiez le chemin vers le dossier NotMNIST
notmnist_folder = 'notMnist_Small'  # Remplacez par votre chemin réel

def load_notmnist(folder):
    image_files = list(Path(folder).rglob("*.png"))
    images = []
    labels = []

    for image_file in image_files:
        try:
            # Chargez l'image en niveaux de gris et redimensionnez-la à 28x28
            image = Image.open(image_file).convert('L').resize((28, 28))
            # Convertissez l'image en tableau NumPy
            image_array = np.array(image)
            # Ajoutez l'image au tableau des images
            images.append(image_array)
            # Ajoutez l'étiquette basée sur le dossier parent
            label = os.path.basename(os.path.dirname(image_file))
            labels.append(ord(label) - ord('A'))  # Convertissez A-J en 0-9
        except Exception as e:
            print(f"Erreur lors du chargement de l'image {image_file}: {e}")

    return np.array(images), np.array(labels)

# Chargez les données NotMNIST
notmnist_images, notmnist_labels = load_notmnist(notmnist_folder)

# Divisez les données en ensembles d'entraînement et de test
train_images, test_images, train_labels, test_labels = train_test_split(
    notmnist_images, notmnist_labels, test_size=0.2, random_state=42
)

# Organisez les données de la même manière que les données MNIST
(train_images, train_labels), (_, _) = (train_images, train_labels), (test_images, test_labels)

train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]



In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256


In [ ]:
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


Le générateur utilise tf.keras.layers.Conv2DTranspose(sachérification) couches pour produire une image à partir d'une graine (bruit aléatoire). Commencer par un Densecouche qui prend cette graine comme entrée, puis absorbe plusieurs fois jusqu'à ce que vous atteigniez la taille d'image souhaitée de 28x28x1. Remarquez le tf.keras.layers.LeakyReLUactivation pour chaque couche, à l'exception de la couche de sortie qui utilise tanh.

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model


Utilisation du générateur (comme non encore entraîné) pour créer une image.

In [ ]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')


Le discriminateur est un classificateur d'image basé sur CNN.

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model


Utiliser le discriminateur (comme non encore formé) pour classer les images générées comme réelles ou fausses. Le modèle sera formé pour produire des valeurs positives pour les images réelles, et des valeurs négatives pour les fausses images.

In [ ]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)


Définir les fonctions de perte et les optimiseurs pour les deux modèles.

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)


Cette méthode permet de quantifier la manière dont le discriminateur est capable de distinguer les images réelles des contrefaçons. Il compare les prédictions du discriminateur sur des images réelles à un tableau de 1s, et les prédictions du discriminateur sur les fausses images (générées) à un tableau de 0s.

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss


La perte du générateur quantifie à quel point il a pu tromper le discriminateur. Intuitivement, si le générateur est performant, le discriminateur classera les fausses images comme réelles (ou 1). Ici, comparer les décisions des discriminateurs sur les images générées à un tableau de 1s.

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


Le discriminateur et les optimiseurs du générateur sont différents puisque vous allez former deux réseaux séparément.

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)


Ce carnet montre également comment sauvegarder et restaurer des modèles, ce qui peut être utile au cas où une longue tâche d'entraînement est interrompue.

In [ ]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 100

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])


La boucle d'entraînement commence par le générateur recevant une graine aléatoire comme entrée. Cette graine est utilisée pour produire une image. Le discriminateur est ensuite utilisé pour classer des images réelles (dessine de l'ensemble d'apprentissage) et des images fausses (produites par le générateur). La perte est calculée pour chacun de ces modèles, et les gradients sont utilisés pour mettre à jour le générateur et le discriminateur.

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


Générer et sauvegarder des images

In [ ]:
def generate_and_save_images(generator, epoch, test_input, notmnist_folder):
    predictions = generator.predict(test_input)
    predictions = 0.5 * predictions + 0.5  # Rénormalisez les valeurs entre [0, 1]

    for i in range(predictions.shape[0]):
        label = chr(ord('A') + i)
        folder_path = os.path.join(notmnist_folder, label + "_generated")
        os.makedirs(folder_path, exist_ok=True)

        image_path = os.path.join(folder_path, f"generated_{epoch}_{i}.png")
        generated_image = Image.fromarray((predictions[i] * 255).astype(np.uint8)[:, :, 0])
        generated_image.save(image_path)


In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)



Appelez le train()Méthode définie ci-dessus pour entraîner simultanément le générateur et le discriminateur. Notez que la formation des GAN peut être délicate. Il est important que le générateur et le discriminateur ne se dominent pas les uns les autres (par exemple, qu'ils s'entraînent à une cadence similaire).

Au début de la formation, les images générées ressemblent à un bruit aléatoire. Au fur et à mesure que la formation progresse, les chiffres générés auront l'air de plus en plus réels. Après environ 50 époques, ils ressemblent à des chiffres MNIST. Cela peut prendre environ une minute / epépoque avec les paramètres par défaut sur Colab.

In [ ]:
 
train(train_dataset, 100)


definition du model pour la classification,il s'agit du meme modele utiliser pour les données originales notMnist 

In [ ]:
# Définir l'architecture du modèle
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(10), activation='softmax')
])
# Compiler le modèle
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


entrainement

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)
# Entraîner le modèle sans augmentation des données
model.fit(train_images, train_labels, batch_size=64, epochs=10, validation_data=(test_images, test_labels))

# Évaluer le modèle sur l'ensemble de test
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Accuracy on test set: {test_acc}')
